In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime
import matplotlib.pyplot as plt
from pprint import pprint

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.model_selection import GroupKFold

import warnings
warnings.filterwarnings("ignore")

2023-05-31 15:50:57.210187: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 15:50:57.237328: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 15:50:57.237959: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 15:50:58.013350: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset = pd.read_csv("../StationsDatasets/dataset.csv").drop(columns=["Unnamed: 0"])

In [3]:
cols = ["StationEnd", "WorkingDay", "Hour", "Count", "Count1", "Count1week", "Count2week", "Count3week", "Temp", "Humidity", "WeatherSituation", "Windspeed"]

epochs = 30
batch_size = 1
verbose = 2

In [4]:
met1 = dataset[[col for col in dataset.columns if col in cols]]

X = met1.drop(columns=["Count"])
y = met1["Count"]

X = np.reshape(X.values, (X.shape[0], 1, X.shape[1]))

# Define the K-fold Cross Validator
kfold = GroupKFold(n_splits=10)

acc_per_fold = []
loss_per_fold = []

model_met1 = Sequential()
model_met1.add(LSTM(4, input_shape=(1,met1.shape[1]-1)))
model_met1.add(Dense(1))
model_met1.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])

# K-fold Cross Validation model evaluation
for train, test in kfold.split(X, y, dataset["StationEnd"]):
    model_met1.fit(X[train], y[train], epochs=epochs, batch_size=batch_size, verbose=verbose)

    # Generate generalization metrics
    scores = model_met1.evaluate(X[test], y[test], verbose=0)
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

2023-05-31 16:50:32.158500: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-31 16:50:32.159197: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/30


2023-05-31 16:50:32.590539: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-31 16:50:32.591789: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-31 16:50:32.592585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

182739/182739 - 163s - loss: 3.9129 - accuracy: 0.3223 - 163s/epoch - 895us/step
Epoch 2/30
182739/182739 - 158s - loss: 3.7029 - accuracy: 0.3230 - 158s/epoch - 865us/step
Epoch 3/30
182739/182739 - 158s - loss: 3.6684 - accuracy: 0.3230 - 158s/epoch - 865us/step
Epoch 4/30
182739/182739 - 159s - loss: 3.6491 - accuracy: 0.3230 - 159s/epoch - 868us/step
Epoch 5/30
182739/182739 - 159s - loss: 3.6293 - accuracy: 0.3229 - 159s/epoch - 868us/step
Epoch 6/30
182739/182739 - 158s - loss: 3.5949 - accuracy: 0.3224 - 158s/epoch - 867us/step
Epoch 7/30
182739/182739 - 160s - loss: 3.5745 - accuracy: 0.3221 - 160s/epoch - 873us/step
Epoch 8/30
182739/182739 - 156s - loss: 3.5588 - accuracy: 0.3223 - 156s/epoch - 854us/step
Epoch 9/30
182739/182739 - 158s - loss: 3.5443 - accuracy: 0.3222 - 158s/epoch - 864us/step
Epoch 10/30
182739/182739 - 158s - loss: 3.5413 - accuracy: 0.3221 - 158s/epoch - 864us/step
Epoch 11/30
182739/182739 - 158s - loss: 3.5322 - accuracy: 0.3219 - 158s/epoch - 864us/st

2023-05-31 18:09:41.811325: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-31 18:09:41.812756: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-31 18:09:41.813773: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/30
184815/184815 - 163s - loss: 3.4986 - accuracy: 0.3324 - 163s/epoch - 881us/step
Epoch 2/30
184815/184815 - 162s - loss: 3.4988 - accuracy: 0.3323 - 162s/epoch - 876us/step
Epoch 3/30
184815/184815 - 162s - loss: 3.4953 - accuracy: 0.3322 - 162s/epoch - 877us/step
Epoch 4/30
184815/184815 - 162s - loss: 3.4977 - accuracy: 0.3323 - 162s/epoch - 876us/step
Epoch 5/30
184815/184815 - 162s - loss: 3.4915 - accuracy: 0.3323 - 162s/epoch - 876us/step
Epoch 6/30
184815/184815 - 162s - loss: 3.4920 - accuracy: 0.3322 - 162s/epoch - 875us/step
Epoch 7/30
184815/184815 - 161s - loss: 3.4975 - accuracy: 0.3323 - 161s/epoch - 873us/step
Epoch 8/30
184815/184815 - 161s - loss: 3.4966 - accuracy: 0.3323 - 161s/epoch - 872us/step
Epoch 9/30
184815/184815 - 161s - loss: 3.4957 - accuracy: 0.3323 - 161s/epoch - 872us/step
Epoch 10/30
184815/184815 - 161s - loss: 3.4948 - accuracy: 0.3323 - 161s/epoch - 873us/step
Epoch 11/30
184815/184815 - 161s - loss: 3.4917 - accuracy: 0.3323 - 161s/epoch